In [1]:
import findspark
# findspark.init('/usr/local/spark')
findspark.init()
# https://github.com/minrk/findspark

import pyspark
import random
try:
    del sc
except:
    pass
sc = pyspark.SparkContext(appName='myTest')

# sc.stop()

https://spark.apache.org/examples.html

https://www.quora.com/What-is-the-Alternating-Least-Squares-method-in-recommendation-systems-And-why-does-this-algorithm-work-intuition-behind-this

https://bugra.github.io/work/notes/2014-04-19/alternating-least-squares-method-for-collaborative-filtering/

https://stanford.edu/~rezab/classes/cme323/S15/notes/lec14.pdf

https://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS

https://www.codementor.io/jadianes/building-a-recommender-with-apache-spark-python-example-app-part1-du1083qbw


In [2]:
import os
import urllib
import zipfile 

# downloading data from web and unzip them to MovieData folder
# https://stackoverflow.com/questions/7243750/download-file-from-web-in-python-3

datasetPath = os.path.join(os.getcwd(), 'Downloads', 'MovieData')

cDatasetPath = os.path.join(datasetPath, 'ml-latest.zip')
sDatasetPath = os.path.join(datasetPath, 'ml-latest-small.zip')

cDatasetUrl = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip'
sDatasetUrl = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'

cDownload = urllib.request.urlretrieve(cDatasetUrl, cDatasetPath)
print('complete data downloaded')
sDownload = urllib.request.urlretrieve(sDatasetUrl, sDatasetPath)
print('small data downloaded')

with zipfile.ZipFile(cDatasetPath, 'r') as z:
    z.extractall(datasetPath)
    
with zipfile.ZipFile(sDatasetPath, 'r') as z:
    z.extractall(datasetPath)

complete data downloaded
small data downloaded


In [3]:
# import findspark
# findspark.init()
# https://github.com/minrk/findspark

# import pyspark
import os

from pyspark.mllib.recommendation import ALS
import math

datasetPath = os.path.join(os.getcwd(), 'Downloads', 'MovieData')

# sc = pyspark.SparkContext(appName='Recommand Movie')
sc = pyspark.SparkContext.getOrCreate()

# process ratings.csv
sRatingsFile = os.path.join(datasetPath, 'ml-latest-small', 'ratings.csv')
sRawData = sc.textFile(sRatingsFile)
sRawDataHeader = sRawData.take(1)[0]
# print(sRawDataHeader)

sRatingsData = sRawData.filter(lambda line: line!=sRawDataHeader)\
                      .map(lambda line: line.split(','))\
                      .map(lambda tokens: (tokens[0], tokens[1], tokens[2]))\
                      .cache()

# print(sRatingsData.take(3))


# process movies.csv
sMoviesFile = os.path.join(datasetPath, 'ml-latest-small', 'movies.csv')
sMoviesRawData = sc.textFile(sMoviesFile)
sMoviesRawDataHeader = sMoviesRawData.take(1)[0]

sMoviesData = sMoviesRawData.filter(lambda line: line!=sMoviesRawDataHeader)\
                      .map(lambda line: line.split(','))\
                      .map(lambda tokens: (tokens[0], tokens[1]))\
                      .cache()

# print(sMoviesData.take(3))


# Collaborative Filtering
# https://stackoverflow.com/questions/32933143/how-does-sparks-rdd-randomsplit-actually-split-the-rdd
# http://www.grasshopper3d.com/forum/topics/what-are-random-seed-values
training_RDD, validation_RDD, test_RDD = sRatingsData.randomSplit([6, 2, 2], seed=0)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

# train the model
seed = 5
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print('For rank %s the RMSE is %s' % (rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print('The best model was trained with rank %s' % best_rank)

print('\nPrediction Model: UserID, the MovieID, and the Rating\n',predictions.take(3))
sc.stop()

For rank 4 the RMSE is 0.9306227802083504
For rank 8 the RMSE is 0.9430857404117656
For rank 12 the RMSE is 0.9444923612758019
The best model was trained with rank 4

Prediction Model: UserID, the MovieID, and the Rating
 [((302, 1084), 3.5142323156707356), ((358, 1084), 3.8431194591428897), ((242, 1084), 4.625278621518187)]


https://datascience.ibm.com/exchange/public/entry/view/99b857815e69353c04d95daefb3b91fa
    IBM explained movie recommandation